In [4]:
from pyro.distributions import Dirichlet
import matplotlib.tri as tri
import torch
import pandas as pd
import numpy as np

sample = torch.from_numpy(np.array([[1/3, 1/3, 1/3], [1/3, 1/3, 1/3], [1/3, 1/3, 1/3]]))



corners = np.array([[0, 0], [1, 0], [0.5, 0.75**0.5]])
AREA = 0.5 * 1 * 0.75**0.5
triangle = tri.Triangulation(corners[:, 0], corners[:, 1])
refiner = tri.UniformTriRefiner(triangle)
trimesh = refiner.refine_triangulation(subdiv=4)

# For each corner of the triangle, the pair of other corners
pairs = [corners[np.roll(range(3), -i)[1:]] for i in range(3)]
# The area of the triangle formed by point xy and another pair or points
tri_area = lambda xy, pair: 0.5 * np.linalg.norm(np.cross(*(pair - xy)))

def xy2bc(xy, tol=1.e-4):
    '''Converts 2D Cartesian coordinates to barycentric.'''
    coords = np.array([tri_area(xy, p) for p in pairs]) / AREA
    return np.clip(coords, tol, 1.0 - tol)


def return_Dirichlet( polygon_points, only_grid_points, samples = sample,concentration = 30):
        new_z_values = []
        # get new value of points in polygon
                # convert to coordinates with 3, rather than to points of reference for Direichlet input
        x,y = map(list,zip(*polygon_points))
        points = torch.tensor(np.array([(xy2bc(xy)) for xy in zip(x, y)]))
        points /= torch.sum(points, dim=1, keepdim=True)

        alpha = samples * concentration
        vals = torch.stack([torch.exp(Dirichlet(alpha).log_prob(points[i,:])) for i in range(points.shape[0])])
        vals /= torch.max(vals, dim=0, keepdim=True)[0]
        vals = torch.sum(vals, dim=1)
        vals /= torch.sum(vals)

        new_z_values = [(x, y, z.item()) for (x, y), z in zip(polygon_points, vals)]
        
        z_poly_list = [0] * len(only_grid_points)
        new_z_values.extend([(x, y, z) for (x, y), z in zip(only_grid_points, z_poly_list)])

        # add in zero for anything in grid not in polygon

        return new_z_values

def example_function(polygon_points, only_grid_points):
    new_z_values = []
    # get new value of points in polygon
    z_poly_list = [2] * len(polygon_points)
    new_z_values = [(x, y, z) for (x, y), z in zip(polygon_points, z_poly_list)]

    z_poly_list = [0] * len(only_grid_points)
    new_z_values.extend([(x, y, z) for (x, y), z in zip(only_grid_points, z_poly_list)])

    # add in zero for anything in grid not in polygon

    return new_z_values

In [105]:

#https://stackoverflow.com/questions/73952977/draw-a-polygon-in-python-using-a-function-fn-r
import math
import matplotlib.pyplot as plt
import numpy as np, pandas as pd, shapely.vectorized
from shapely.geometry import Polygon
import json
import vega
import torch

#https://stackoverflow.com/questions/23411688/drawing-polygon-with-n-number-of-sides-in-python-3-2
def polygon(sides, radius=5, rotation=0, translation=(5, 5)):
    one_segment = math.pi * 2 / sides

    points = [
        (math.sin(one_segment * i + rotation) * radius,
         math.cos(one_segment * i + rotation) * radius)
        for i in range(sides)]

    if translation:
        points = [[sum(pair) for pair in zip(point, translation)]
                  for point in points]

    return points


# https://stackoverflow.com/questions/73693639/python-plot-a-regular-grid-of-points-in-a-polygon
def points_in_polygon(polygon_geom):
    # define grid spec from min to max value with spacing of grid_res=10
    grid_res = 1

    # this isn't necessary in your case, but if your polygon had
    # boundaries not falling on grid edges, you might want to ensure
    # the grid started on specific cell boundaries or midpoints
    minx = np.floor(polygon_geom.bounds[0] / grid_res) * grid_res
    maxx = np.ceil(polygon_geom.bounds[2] / grid_res) * grid_res
    miny = np.floor(polygon_geom.bounds[1] / grid_res) * grid_res
    maxy = np.ceil(polygon_geom.bounds[3] / grid_res) * grid_res

    # you can form the grid by setting up the coordinates and then filling
    # the grid with numpy.meshgrid
    x = np.arange(minx, maxx + grid_res / 2, grid_res)
    y = np.arange(miny, maxy + grid_res / 2, grid_res)
    XX, YY = np.meshgrid(x, y)

    # use shapely.vectorized to find all points within the polygon
    in_polygon = shapely.vectorized.contains(polygon_geom, XX, YY)

    # filter the points and flatten them to 1D vectors
    x_in_polygon = XX[in_polygon].ravel()
    y_in_polygon = YY[in_polygon].ravel()

    x_not_in_polygon = XX[~in_polygon].ravel()
    y_not_in_polygon = YY[~in_polygon].ravel()
    return list(zip(x_in_polygon, y_in_polygon)), list(zip(x_not_in_polygon, y_not_in_polygon))

def xy2bc(xy, tol=1.e-4):
    '''Converts 2D Cartesian coordinates to barycentric.'''
    coords = np.array([tri_area(xy, p) for p in pairs]) / AREA
    return np.clip(coords, tol, 1.0 - tol)



# return square grid, 0 for not points in polygon
def create_polygon_grid(n = 4, r = 25, my_function = example_function):
    # get coordinates of polygon in x and y
    points = polygon(sides = n, radius = r, translation=(r, r))

    # get grid 
    polygon_points, only_grid_points = points_in_polygon(Polygon(points))

    # return grid with values for shape and 0 for grid outside of shape
    new_z_values = my_function(polygon_points, only_grid_points)

    df =  pd.DataFrame.from_records(new_z_values, columns=['x', 'y', 'val'])
    df = df.sort_values(['y', 'x'], ascending = [False, True])
    
    width =   df.x.min()
    height =  df.y.max()  


    initial_point = ['%s%d %d' % ('L', x - width, height - y) for i, (x, y) in enumerate(points)][0]
    total = ['%s%d %d' % (['M', 'L'][i>0], x - width, height - y) for i, (x, y) in enumerate(points)]
    total.append(initial_point)
   # https://www.petercollingridge.co.uk/tutorials/python/converting-python-list-svg-path/
   # don't know why the - width is necessary, will need to look into it
    d = ' '.join( total)
    print(d)
    # return json file with height, weight, values in grid, and xy values for svg path
    json_dict ={}
    json_dict['width'] = df.x.max()  + 1 - df.x.min()
    json_dict['height'] = df.y.max()  + 1 - df.y.min()
    json_dict['values'] = df['val'].tolist()
    json_dict['svg_path'] = d

    return json.loads(json.dumps(json_dict))




In [111]:

# change r to get new polygon

json_df = create_polygon_grid(n =20, r = 100)

with open("contour_grid.json", "w") as outfile:
    outfile.write(json.dumps(json_df))






vega.ipy_display(vega.polygon_contour_vega(json_df))

M100 0 L130 4 L158 19 L180 41 L195 69 L200 100 L195 130 L180 158 L158 180 L130 195 L100 200 L69 195 L41 180 L19 158 L4 130 L0 100 L4 69 L19 41 L41 19 L69 4 L100 0


In [107]:

# change r to get new polygon

json_df = create_polygon_grid(n =3, r = 100, my_function=return_Dirichlet)

with open("contour_grid.json", "w") as outfile:
    outfile.write(json.dumps(json_df))






vega.ipy_display(vega.polygon_contour_vega(json_df))

M87 0 L173 149 L0 150 L87 0
